# Librerias

In [1]:
import pandas as pd
import base64
import requests
import datetime
from urllib.parse import urlencode
import json
import re

# Coneccion con la API de Spotify

In [8]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    client_id = None
    client_secret = None
    access_token_did_expire = True
    token_url = 'https://accounts.spotify.com/api/token'
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args,**kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
    
    # Aqui se crean las credenciales en base 64 para poder acceder a la API
    def get_client_credentials(self):
        """ returns a base64 encoded string """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
            
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())        
        return client_creds_b64.decode()
    
    # Aqui estamos generando los headers de acceso
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        
        return {
            "Authorization":f"Basic {client_creds_b64}"
        }
    
    # Aqui obtenemos el Token Data
    def get_token_data(self):
        return {
            "grant_type":"client_credentials"
        }
    
    # Aqui nos autentificamos y obtenemos el token de acceso
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        
        r = requests.post(token_url, data = token_data, headers = token_headers)

        if r.status_code not in range(200,299):
            raise Exceptiontion('Could not authenticate client')
            
        data = r.json()
        now = datetime.datetime.now()
        acces_token = data['access_token']
        expires_in = data['expires_in'] #seconds
        expires = now + datetime.timedelta(seconds = expires_in)
        self.access_token = acces_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    # Extraemos el token de acceso
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token()
        return token
    
    # Aqui son los headers de para las busquedas
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization":f"Bearer {access_token}"
        }
        return headers
        
    # busqueda de listas de reproduccion
    def get_user_playlist(self,u_id):
        endpoint = f'https://api.spotify.com/v1/users/{u_id}/playlists'
        headers = self.get_resource_header()
        r = requests.get(endpoint,headers=headers)
        if r.status_code not in range(200,299):
            return {}
        return r.json()
    
    # Busqueda de las canciones por playlist: nombre, 
    def get_song_playlist(self,playlist_id):
        endpoint = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks'
        headers = self.get_resource_header()
        r = requests.get(endpoint,headers=headers)
        if r.status_code not in range(200,299):
            return {}
        return r.json()
    
    # Busqueda de las caracteristicas de las canciones
    def get_audio_features(self,song_id):
        endpoint = f'https://api.spotify.com/v1/audio-features/{song_id}'
        headers = self.get_resource_header()
        r = requests.get(endpoint,headers=headers)
        if r.status_code not in range(200,299):
            return {}
        return r.json()
    
    # Busqueda de la informacion de los artistas
    def get_data_artist(self,artist_id):
        endpoint = f'https://api.spotify.com/v1/artists/{artist_id}'
        headers = self.get_resource_header()
        r = requests.get(endpoint,headers=headers)
        if r.status_code not in range(200,299):
            return {}
        return r.json()
    
    def recommendation_genre_seeds(self):
        """ Get a list of genres available for the recommendations function.
        """
        endpoint = f'https://api.spotify.com/v1/recommendations/available-genre-seeds'
        headers = self.get_resource_header()
        r = requests.get(endpoint,headers=headers)
        if r.status_code not in range(200,299):
            return {}
        return r.json()
    
    # Seguir revisando funcion
    def get_track_recommendations(self):
        # Pendiente modificar el endpoint con los parametros de busqueda
        endpoint = "https://api.spotify.com/v1/recommendations?limit=20&seed_artists=4NHQUGzhtTLFvgF5SZesLK&seed_genres=rock&seed_tracks=0c6xIDDpzE81m2q797ordA%2C7ne5aZDaAUnundo6aI2KOG%2C2wOMnyjpDSU20v9fwLFITn&min_valence=.5&max_valence=1&target_valence=.8"
        headers = self.get_resource_header()
        r = requests.get(endpoint,headers=headers
        if r.status_code not in range(200,299):
            return {}
        return r.json()

    # https://github.com/plamere/spotipy/blob/master/spotipy/client.py --> Codigo Fuente
    # https://spotipy.readthedocs.io/en/2.17.1/ --> Libreria
    def recommendations(self,seed_artists=None,seed_genres=None,seed_tracks=None,limit=20,country=None,**kwargs):
        """ Get a list of recommended tracks for one to five seeds.
            (at least one of `seed_artists`, `seed_tracks` and `seed_genres`
            are needed)
            Parameters:
                - seed_artists - a list of artist IDs, URIs or URLs
                - seed_tracks - a list of track IDs, URIs or URLs
                - seed_genres - a list of genre names. Available genres for
                                recommendations can be found by calling
                                recommendation_genre_seeds
                - country - An ISO 3166-1 alpha-2 country code. If provided,
                            all results will be playable in this country.
                - limit - The maximum number of items to return. Default: 20.
                          Minimum: 1. Maximum: 100
                - min/max/target_<attribute> - For the tuneable track
                    attributes listed in the documentation, these values
                    provide filters and targeting on results.
        """
        params = dict(limit=limit)
        if seed_artists:
            params["seed_artists"] = ",".join(
                [self._get_id("artist", a) for a in seed_artists]
            )
        if seed_genres:
            params["seed_genres"] = ",".join(seed_genres)
        if seed_tracks:
            params["seed_tracks"] = ",".join(
                [self._get_id("track", t) for t in seed_tracks]
            )
        if country:
            params["market"] = country

        for attribute in [
            "acousticness",
            "danceability",
            "duration_ms",
            "energy",
            "instrumentalness",
            "key",
            "liveness",
            "loudness",
            "mode",
            "popularity",
            "speechiness",
            "tempo",
            "time_signature",
            "valence",
        ]:
            for prefix in ["min_", "max_", "target_"]:
                param = prefix + attribute
                if param in kwargs:
                    params[param] = kwargs[param]
        return self._get("recommendations", **params)
    

In [3]:
# Llaves de acceso para conectarnos a la API de Spotify
client_id = 'SECRETO'
client_secret = 'SECRETO'

In [4]:
# Llamamos a la funcion y le asignamos nuestros client ID y Client Secret
spotify = SpotifyAPI(client_id,client_secret)

In [5]:
# Nos indica si fue realizada la coneccion con la API
spotify.perform_auth()

True

In [6]:
# Obtenemos el access_token que nos servirá para las consultas nescesarias. Por eso lo guardamos en una variable y que sea mas
# sencillo llamarlo posteriormente.
access_token = spotify.access_token
access_token

'BQA7p2MvXRyPfuxVaUIIEUkwxQllFkp4hzcAtFdUdE9C7634Zr-YoYDcYeTOMO-ErfmOPHa903g5ifzi5rQ'

Valence: Medida de 0.0 a 1.0 que describe la positividad musical que transmite una pista. Las pistas con valencia alta 
suenan más positivas (por ejemplo, feliz, alegre, eufórico), mientras que las pistas con valencia baja suenan más 
negativas (por ejemplo, triste, deprimido, enojado).

# Obtencion de la Data

In [16]:
#https://medium.com/@goyoregalado/bots-de-telegram-en-python-134b964fcdf7
# Bot de Telegram

# Regex Para Identificar Lista de Reproduccion o Canciones

In [20]:
def get_id_playlist(url):
    """
    Se recibe un link a la lista de reproduccion, y se extrae unicamente el ID de dicha lista 
    para analizar las canciones y sus atributos
    """
    expresion = r'\w{22,23}'
    id_playlist = re.findall(expresion,url)
    if len(id_playlist)>1:
        return id_playlist[0]
    else:
        return id_playlist

def get_id_artist(url):
    """
    Se recibe un link del artista, y se extrae unicamente el ID de dicho artista para analizar las canciones
    y sus atributos
    """
    expresion = r'\w{22,23}'
    id_artist = re.findall(exprecion,url)
    if len(id_artist)>1:
        return id_artist[0]
    else:
        return id_artist

def song_playlist(id_playlist):

    """
    Funcion para obtener los ID de canciones que se encuentran en una Playlist. 
    Retorna una lista con los ID
    """
    canciones = spotify.get_song_playlist(id_playlist)
    id_canciones = pd.json_normalize(canciones['items'])
    song_list = id_canciones['track.id'].to_list()
    
    return song_list

def song_features(canciones_id):
    """
    Funcion para obtener las caracteristicas de las canciones. 
    descarga y limpia los datos.
    Requiere de un listado de ID de canciones
    Nos retorna un DataFrame con las caracteristicas de las canciones.
    """
    # Hacemos el DataFrame de todas las caracteristicas de las canciones obtenidas de las Playlist
    song_features = []
    # Buscamos la info de cada cancion
    for i in canciones_id:
        features = spotify.get_audio_features(i)
        features_norm = pd.json_normalize(features)
        #featrures_norm['ID Cancion'] = i # aqui agregamos la columna con el numero de ID de la cancion
        # Creamos una lista con la info por cada cancion
        song_features.append(features_norm)
    
    # Generamos un solo DF
    df_features = pd.concat(song_features)
    # Eliminamos las columnas inservibles
    drop_cols =['type','uri','track_href', 'analysis_url', 'duration_ms','time_signature']
    df_features= df_features.drop(columns=drop_cols,axis=1).reset_index(drop=True)
    
    return df_features

# Obtencion de la informacion por parte del usuario
id_artist = []
id_playlist = []
no_viable = []

# se le pide al usuario la info separada por comas
user_input = input().split(',')

# se identifica si representa a un artista o playlist
for i in user_input:
    if 'artist' in i:
        id_artist.append(i)
    elif 'playlist' in i:
        id_playlist.append(i)
    else:
        no_viable.append(i)

# Pasamos la lista de URL y dejamos unicamente los ID buscados
id_playlist = [get_id_playlist(url) for url in id_playlist] 
id_artist = [get_id_artist(url) for url in id_artist]

print('ID Playlist: ',id_playlist)
print('ID Artist: ',id_artist)

https://open.spotify.com/playlist/6Fc15OjwTbMpKbMen7jcfV?si=YOb_KgcyTNmvYJjwpaOfrQ
ID Playlist:  ['6Fc15OjwTbMpKbMen7jcfV']
ID Artist:  []


In [ ]:
# Listas de reproduccion
Listas_reproduccion = ['https://open.spotify.com/playlist/6Fc15OjwTbMpKbMen7jcfV?si=YOb_KgcyTNmvYJjwpaOfrQ','https://open.spotify.com/playlist/37i9dQZF1DX95gx8SY6DLX?si=Yc6aodgDSBCZEAFhkLAP8w']

In [ ]:
# Artistas
Lista_artistas = ['https://open.spotify.com/artist/6olE6TJLqED3rqDCT0FyPh?si=qqDe2hHFTbSlxbMoIkbwhg','https://open.spotify.com/artist/2QsynagSdAqZj3U9HgDzjD?si=fXrkOyVMRxGBDbxJOdv26A']

In [ ]:
#https://developer.spotify.com/console/get-recommendations/?seed_artists=4NHQUGzhtTLFvgF5SZesLK&seed_tracks=0c6xIDDpzE81m2q797ordA&min_energy=0.4&min_popularity=50&market=US

# Pruebas recomendacion

In [26]:
canciones = song_playlist('6Fc15OjwTbMpKbMen7jcfV')
caracteristicas = song_features(canciones)

In [27]:
caracteristicas

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id
0,0.298,0.990,4,-3.357,0,0.1280,0.00873,0.000166,0.4720,0.455,134.754,7ne5aZDaAUnundo6aI2KOG
1,0.723,0.885,6,-4.512,0,0.0429,0.06080,0.000000,0.0899,0.602,131.968,0HCppMSh838nCgSxskJGzz
2,0.364,0.880,7,-1.668,1,0.0491,0.23100,0.000019,0.1480,0.261,141.910,2wOMnyjpDSU20v9fwLFITn
3,0.338,0.436,4,-3.502,1,0.0355,0.88600,0.000000,0.0964,0.286,180.028,5vkO24j51l4ApsHE7Gw9nK
4,0.471,0.366,4,-13.938,1,0.0395,0.00368,0.000000,0.1100,0.273,142.646,1dU383l2TLND7OfsFzSbrE
...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.578,0.803,1,-7.576,0,0.0285,0.01730,0.000000,0.1710,0.723,139.991,4SeWTbfjNvu6ljXCo61tra
96,0.451,0.463,7,-10.464,1,0.0281,0.34500,0.000000,0.1110,0.228,130.244,4Bc6LZXRkPZ2NWDDS1GTbf
97,0.453,0.820,0,-6.184,1,0.0699,0.09710,0.000000,0.0560,0.641,158.635,3w4AROEjy3xNq7hBvKXbN0
98,0.618,0.859,4,-4.057,1,0.0682,0.01310,0.000007,0.2050,0.495,131.952,2CBofenBewzkNUKyxDZOnO


In [61]:
canciones

seed_tracks = ['0c6xIDDpzE81m2q797ordA','7ne5aZDaAUnundo6aI2KOG','2wOMnyjpDSU20v9fwLFITn']
seed_artist = '4NHQUGzhtTLFvgF5SZesLK'
seed_genres = 'country'

recomendaciones = spotify.get_track_recommentations(seed_tracks=seed_tracks,seed_artist=seed_artist,seed_genres=seed_genres)

In [62]:
# PREGUNTAR POR EL ARTISTA FAVORITO, LISTA DE REPRODUCCION Y GENEROS FAVORITOS
recomendaciones

{}

In [57]:
seed_tracks_url = ''
for seed_track in seed_tracks:
    seed_tracks_url += seed_track + '%2C'

seed_tracks_url = seed_tracks_url[:-3]


In [59]:
original = '0c6xIDDpzE81m2q797ordA%2C7ne5aZDaAUnundo6aI2KOG%2C2wOMnyjpDSU20v9fwLFITn'
seed_tracks_url==original

True

In [7]:
spotify.recommendation_genre_seeds()

{'genres': ['acoustic',
  'afrobeat',
  'alt-rock',
  'alternative',
  'ambient',
  'anime',
  'black-metal',
  'bluegrass',
  'blues',
  'bossanova',
  'brazil',
  'breakbeat',
  'british',
  'cantopop',
  'chicago-house',
  'children',
  'chill',
  'classical',
  'club',
  'comedy',
  'country',
  'dance',
  'dancehall',
  'death-metal',
  'deep-house',
  'detroit-techno',
  'disco',
  'disney',
  'drum-and-bass',
  'dub',
  'dubstep',
  'edm',
  'electro',
  'electronic',
  'emo',
  'folk',
  'forro',
  'french',
  'funk',
  'garage',
  'german',
  'gospel',
  'goth',
  'grindcore',
  'groove',
  'grunge',
  'guitar',
  'happy',
  'hard-rock',
  'hardcore',
  'hardstyle',
  'heavy-metal',
  'hip-hop',
  'holidays',
  'honky-tonk',
  'house',
  'idm',
  'indian',
  'indie',
  'indie-pop',
  'industrial',
  'iranian',
  'j-dance',
  'j-idol',
  'j-pop',
  'j-rock',
  'jazz',
  'k-pop',
  'kids',
  'latin',
  'latino',
  'malay',
  'mandopop',
  'metal',
  'metal-misc',
  'metalcore',
